In [1]:
from gradio_client import Client

client = Client("https://waleko-segmentanythingxgroundingdino.hf.space/")
result = client.predict(
				"https://raw.githubusercontent.com/gradio-app/gradio/main/test/test_files/bus.png",	# str representing filepath or URL to image in 'Upload Image' Image component
				"Howdy!",	# str representing string value in 'Object to Detect' Textbox component
				api_name="/predict"
)
print(result)

/home/amir/miniconda3/envs/image-assess/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded as API: https://waleko-segmentanythingxgroundingdino.hf.space/ ✔
('/tmp/gradio/tmp7f80mb8g.json', '/tmp/gradio/0d0dab98-8cb9-41e2-9f78-0ec02a9e6c24')


In [2]:
!ls /tmp/gradio/

0d0dab98-8cb9-41e2-9f78-0ec02a9e6c24  tmp7f80mb8g.json


# Prototype

In [5]:
prompt = "aminal"   # prompt for text model
result_dict = {}

## Download images

In [6]:
# load autoreload extension
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from downloader import download_frames_from_video
import os

stream = "https://www.youtube.com/watch?v=g_L1Ay8P244"
parsed_pic_dir = "sloth_tv"
os.makedirs(parsed_pic_dir, exist_ok=True)
download_frames_from_video(stream, parsed_pic_dir, every_nth=24, amount=2, verbose=True)

15:25:05 ::    Helper     :: WARNING  :: GStreamer not found!
15:25:05 ::    Helper     ::  DEBUG   :: Selecting `best` resolution for streams.
15:25:05 ::    CamGear    ::   INFO   :: Verifying Streaming URL using yt-dlp backend. Please wait...


15:25:07 ::    CamGear    ::   INFO   :: [Backend] :: Streaming URL is fully supported. Available Streams are: [144p, 240p, 360p, 480p, 720p, 1080p, best, worst]
15:25:07 ::    CamGear    :: WARNING  :: Livestream URL detected. It is advised to use GStreamer backend(`cv2.CAP_GSTREAMER`) with it.
15:25:07 ::    CamGear    ::  DEBUG   :: Using `best` resolution for streaming.
15:25:07 ::    CamGear    ::  DEBUG   :: YouTube source ID: `g_L1Ay8P244`, Title: `Sloth TV powered by EXPLORE.org 2023-11-07 15:25`, Quality: `best`
15:25:07 ::    CamGear    ::  DEBUG   :: Enabling Threaded Queue Mode for the current video source!
15:25:11 ::    CamGear    ::  DEBUG   :: Terminating processes.


## Segmentating photos

In [31]:
from gradio_client import Client

client = Client("https://waleko-segmentanythingxgroundingdino.hf.space/")

result_dict["photos"] = os.listdir(parsed_pic_dir)
result_dict["seg_predictions"] = []

for img in result_dict["photos"]:
    result_dict["seg_predictions"].append(client.predict(
        os.path.join(parsed_pic_dir, img),	# str representing filepath or URL to image in 'Upload Image' Image component
        prompt,	# str representing string value in 'Object to Detect' Textbox component
        api_name="/predict"
    ))

Loaded as API: https://waleko-segmentanythingxgroundingdino.hf.space/ ✔


## Assessing photos

In [38]:
result_dict["seg_predictions"][2][0], result_dict["seg_predictions"][2][1], os.listdir(result_dict["seg_predictions"][2][1])

('/tmp/gradio/tmpt9y7t9hq.json',
 '/tmp/gradio/67e56782-7b67-440a-bd9d-8ef9a8d51439',
 ['captions.json', 'dd09de001f95563a100293dc6ee14d5d5cdcaa9d'])

In [46]:
from image_assessment import model_api
from PIL import Image


result_dict["img_score"] = []
result_dict["subimg_scores"] = []
photo_sizes = []
for i in range(len(result_dict["photos"])):

    img = result_dict["photos"][i]
    im = Image.open(os.path.join(parsed_pic_dir, img))
    width, height = im.size
    photo_sizes.append(width*height)

    img_score, _ = model_api.predict(os.path.join(parsed_pic_dir, img))
    result_dict["img_score"].append(img_score)

    res_path = result_dict["seg_predictions"][i][1]
    files = []
    for root, dirs, f in os.walk(res_path):
        for file in f:
            if file.endswith(".jpeg"):
                files.append(os.path.join(root, file))
    print(files)
    sub_scores = []
    for file in files:
        sub_score, _ = model_api.predict(file)
        sub_scores.append(sub_score)
    result_dict["subimg_scores"].append(sub_scores)


/home/amir/miniconda3/envs/image-assess/lib/python3.8/site-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[]
[]
['/tmp/gradio/67e56782-7b67-440a-bd9d-8ef9a8d51439/dd09de001f95563a100293dc6ee14d5d5cdcaa9d/output_image_0.jpeg']


In [47]:
# animal fraction computing
import json


fractions = []
for i in range(len(result_dict["photos"])):
    json_path = result_dict["seg_predictions"][i][0]
    # read json file
    with open(json_path, 'r') as f:
        meta_info = json.load(f)

    print(meta_info)
    fract = 0
    for obj in meta_info["objects"]:
        area = abs((obj["box"][0] - obj["box"][1])*(obj["box"][2] - obj["box"][3]))

        fract += obj["dino_score"]*area/photo_sizes[i]
    fractions.append(fract)
result_dict["fractions"] = fractions

{'prompt': 'aminal', 'objects': []}
{'prompt': 'aminal', 'objects': []}
{'prompt': 'aminal', 'objects': [{'label': 'aminal', 'dino_score': 0.6490511894226074, 'sam_score': 0.9997274279594421, 'box': [504, 43, 1097, 509], 'center': [800.5, 276.0], 'avg_size': 529.5}]}


In [48]:
import pandas as pd

df = pd.DataFrame(result_dict)
df

,photos,seg_predictions,img_score,subimg_scores,fractions
0,frames0.jpg,"(/tmp/gradio/tmp47pp9f84.json, /tmp/gradio/106...",5.860427,[],0.000000
1,frames24.jpg,"(/tmp/gradio/tmpbmqzh78k.json, /tmp/gradio/5dc...",6.284785,[],0.000000
2,photo_2023-10-24_20-36-57.jpg,"(/tmp/gradio/tmpt9y7t9hq.json, /tmp/gradio/67e...",6.273122,[6.41249418258667],0.270756


In [66]:
filtered = df[df["subimg_scores"].apply(len) > 0].copy()
filtered["avg_subimg_score"] = filtered["subimg_scores"].apply(lambda x: sum(x)/len(x))
filtered

,photos,seg_predictions,img_score,subimg_scores,fractions,avg_subimg_score
2,photo_2023-10-24_20-36-57.jpg,"(/tmp/gradio/tmpt9y7t9hq.json, /tmp/gradio/67e...",6.273122,[6.41249418258667],0.270756,6.412494


In [68]:
filtered["final_score"] = filtered["img_score"] + filtered["fractions"]*20 + filtered["avg_subimg_score"]

In [69]:
filtered.sort_values(by="final_score", ascending=False)

,photos,seg_predictions,img_score,subimg_scores,fractions,avg_subimg_score,final_score
2,photo_2023-10-24_20-36-57.jpg,"(/tmp/gradio/tmpt9y7t9hq.json, /tmp/gradio/67e...",6.273122,[6.41249418258667],0.270756,6.412494,18.100729
